# Understand Challenge Set Distribution

Determine the distribution of playlist data attributes in the recsys18 challenge dataset.

Need to know what the profile of unknown playlists are so we can build a representantive dataset 
from the MPD training set.

The challenge data set is 10x1000 playslists.
There is no info header describing the file as in the mpd slice files, just the playlist dictionary.
Each playlist has a tracks dictionary that may be empty.
Each playlist has a 'num_samples: "x"' parameter where x is 0,5,10,25,100 to match the challenge category.
For the title-only playists 'num_samples: "0"' and there are no tracks in tracks dictionary.
For the title a no-title variations there is either a title attribute or there is not.
The num_samples + num_holdouts = num_tracks, which is the lenght of the orignal playlist.
The track entries lists tracks as in the original mpd data set and includes position.
Position is always 0 through num_samples-1 for the sequential seeds.
Position appears to be 0 through num_tracks with some number of drop-outs for the random samples.
That is, the sampling appears to drop out the tracks throughout the playlist.
This means the recommenders should have a rich embedding and are just filling in small holes.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime

In [ ]:
import pandas as pd
import numpy as np

## Use Pandas to Explore Challenge Structure

Pandas provides lots convenience routines and data frame that is easily constructed and queried.

Follow this recipe to load the data and split the results into a table for playlists and tracks.
https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8

In [ ]:
# load data using Python JSON module
with open('data/challenge_set.json','r') as f:
    data = json.loads(f.read())

In [ ]:
# Flatten data
playlists = pd.json_normalize(data, record_path=['playlists'])

In [ ]:
playlists

In [ ]:
tracks = pd.json_normalize(data['playlists'], record_path=['tracks'], meta=['pid'])

In [ ]:
tracks

## Gather stats on each challenge subtype

### Title Only

The source playlist lenth of the title only seeds range from 10-50 tracks, with each length represented fairly evenly.  

In [ ]:
playlists[0:1000].hist( column="num_tracks", bins=41)

In [ ]:
playlists[0:1000].num_tracks.sort_values()

In [ ]:
len(playlists[0:1000].num_tracks.sort_values())

In [ ]:
np.sort(playlists[0:999].num_tracks.unique())

In [ ]:
len(np.sort(playlists[0:1000].num_tracks.unique()))

In [ ]:
histvals = playlists[0:1000].num_tracks.value_counts(bins=40) #.plot(kind='hist')

In [ ]:
histvals

In [ ]:
histvals.hist(bins=40)

### 5 Seed with Title

In [ ]:
playlists[1000:1001]

In [ ]:
playlists[1000:2000].hist( column="num_tracks", bins=100)

In [ ]:
playlists[1000:2000].num_tracks.max()

### 5 Seed without Title

In [ ]:
playlists[2000:2001]

In [ ]:
playlists[2000:3000].hist( column="num_tracks", bins=100)

In [ ]:
playlists[2000:3000].num_tracks.max()

### 10 Seed with Title

In [ ]:
playlists[3000:3001]

In [ ]:
playlists[3000:4000].hist( column="num_tracks", bins=100)

### 10 Seed without Title

In [ ]:
playlists[4000:4001]

In [ ]:
playlists[4000:5000].hist( column="num_tracks", bins=100)

### 25 Seed with Title

In [ ]:
playlists[5000:5001]

In [ ]:
tracks[tracks.pid == 1000001].head()

In [ ]:
playlists[5000:6000].hist( column="num_tracks", bins=100)

### 25 Rand with Title

In [ ]:
playlists[6000:6001]

In [ ]:
tracks[tracks.pid == 1007147].head()

In [ ]:
playlists[6000:7000].hist( column="num_tracks", bins=100)

### 100 Seed with Title

In [ ]:
playlists[7000:7001]

In [ ]:
tracks[tracks.pid == 1010382].head()

In [ ]:
playlists[7000:8000].hist( column="num_tracks", bins=100)

### 100 Rand with Title

In [ ]:
playlists[8000:8001]

In [ ]:
tracks[tracks.pid == 1018569].head()

In [ ]:
playlists[8000:8999].hist( column="num_tracks", bins=100)

### 1 Seed with Title

In [ ]:
playlists[9000:9001]

In [ ]:
playlists[9000:9999].hist( column="num_tracks", bins=100)

In [ ]:
playlists[9000:9999]

### Total num_tracks distribution across all challenge playlist

In [ ]:
playlists[0:10000].hist( column="num_tracks", bins=240)

In [ ]:
np.sort(playlists[0:10000].num_tracks.unique())

In [ ]:
len(np.sort(playlists[0:10000].num_tracks.unique()))

In [ ]:
playlists[0:10000].num_tracks.value_counts()

## Conclusions

- the distribution for 0-seed, 5-seed-title, and 1-seed are all unique.
- the distribution for 5-notitle, 10-seed, and 10-notitle are roughly the same.
- the distribution for 25-seed and 25-rand are the same
- the distribution for 100-seed and 100-rand are the same

need to show the histograms in a 5x2 image plot.

but i can use these counts and the range of min max playlists in each of categories to select random playlists.

rather than making this super difficult I could just sum up all the distributions and select an equivalent count for each num_tracks.
then i can split those up according to the distribution overall.

can also compare how the challenge distribution compares to the entire mpd.

now that i have the challenge data set analyzed need to get back to parsing the full mpd.
how best to pull out my 10k test set from the 1mil list?

I can select the random filter method i used before to get a much smaller training set.
but this is a bit different, i want to keep selecting random from the values that appear in the challenge set.

looking at the unique num_tracks, each num_tracks from 10-250 is represented.
just at different selection densities.
so if i just randomly select according to the count of the distribution i should be good.


